In [ ]:
%load_ext autoreload
%autoreload 2

import pandas_profiling
%pylab inline

import pandas as pd
import geopandas as gp

import seaborn as sns
sns.set(color_codes=True)
import numpy as np

from pathlib import Path
import itertools
from tqdm import tqdm

from analysis.settings.settings import DevelopmentConfig
c = DevelopmentConfig

from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon
import shapely

# open data integration
Cleaning shape files delivered as open data from http://www.laerminfo.at/laermkarten/methoden/inspire.html

> an accompanying blog post can be found at: https://georgheiler.com/2019/02/08/noise-pollution-data-cleanup/

In [ ]:
%ls ../data/

In [ ]:
def iter_dirs(directory_in_str, glob):
    pathlist = Path(directory_in_str).glob(glob)
    for path in pathlist:
        yield str(path)
        
def parse_attributes_from_path(path):
    file_name = path.split('/')[-1]
    elements = file_name.split('_')
    result = {}
    result['year'] = elements[0]
    result['kind'] = elements[1]
    result['timing'] = elements[2]
    result['state'] = elements[-1].split('.')[0]
    return result

def add_columns_to_df(df, items):
    for key, value in items.items():
        df[key] = value
    return df

In [ ]:
paths = iter_dirs(c.BASE_PATH, '**/*.shp')
#items = 2
#print(f"WARN UNSET THIS FOR PRODUCTION. currently only {items} are processed")
#paths = itertools.islice(paths, items)
tmp_appended_data = []
for path in tqdm(paths):
    attributes_from_filenname = parse_attributes_from_path(path)
    df = gp.read_file(path)
    df = add_columns_to_df(df, attributes_from_filenname)
    tmp_appended_data.append(df)

In [ ]:
df = pd.concat(tmp_appended_data, axis=0, sort=False)
df = df.reset_index(drop=True)
display(df.head())
df.shape

In [ ]:
df.crs = {'init' :'epsg:31287'}

## Write output files
Geopackage is a nice standard and a bit more structured than CSV. But CSV is more versatile i.e. as no spatial index and type information are present polygon and multipolygons can be put into the same column. That's why we need to convert geometries to the same type as outlined below.

### CSV
is a bit smaller in file size and has no spatial index / less optimal compared to gpkg but more flexible

In [ ]:
df.to_csv(c.BASE_PATH + 'noise_pollution.gzip.csv', index=False, compression='gzip')


### parquet
A further option is to store as parquet files. They do not support a binary geometry. But can easily be loaded to parall programs like spark.

> NOTE this time also a reprojection to WGS84 is applied

In [ ]:
df = df.to_crs({'init': 'epsg:4326'})

In [ ]:
df['geometry'] = df['geometry'].apply(lambda g: g.wkt)

In [ ]:
df.to_parquet(c.BASE_PATH + 'noise_pollution.parquet', compression='gzip', partition_cols=['kind', 'state', 'timing'])

### GPKG

The cooridnate system is `EPSG:31287`, http://spatialreference.org/ref/epsg/mgi-austria-lambert-2/ it can be stored in the geodatabase for easier downstream processing.

Convert to all multipolygon shapes to support a single type of geometry in gpkg

In [ ]:
def convert_polygon_to_multipolygon(raw_geometry):
    if(isinstance(raw_geometry, shapely.geometry.polygon.Polygon)):
        return MultiPolygon([raw_geometry])
    else:
        # we currently only have MULTIPOLYGON and POLYGON so plain else is good enough
        return raw_geometry
            
df.geometry = df.geometry.apply(convert_polygon_to_multipolygon)
df.head()

In [ ]:
df.to_file(c.BASE_PATH + 'noise_pollution.gpkg', driver="GPKG")